# Lab | Making predictions with logistic regression

In this lab, you will be using the [Sakila](https://dev.mysql.com/doc/sakila/en/) database of movie rentals.

In order to optimize our inventory, we would like to know which films will be rented. We are asked to create a model to predict it. So we use the information we have from May 2005 to create the model.

### Instructions

1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features (X). 
2. Create a query to get the list of all unique film titles and a boolean indicating if it was rented (rental_date) in May 2005. (Create new column called - 'rented_in_may'). This will be our **TARGET** (y) variable.
3. Read the data into a Pandas dataframe.  At this point you should have 1000 rows.  Number of columns depends on the number of features you chose.
4. Analyze extracted features (X) and transform them. You may need to encode some categorical variables, or scale numerical variables.
5. Create a logistic regression model to predict 'rented_in_may' from the cleaned data.
6. Evaluate the results.

In [1]:
# import modules and get pwd

import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


The features retrieved below will form (X) for our model training set

In [2]:
# retrive and frame (X) data from sql

connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'

engine = create_engine(connection_string)

query = '''SELECT f.film_id, f.title, f.rental_rate, f.special_features, f.rental_duration, f.length, f.rating,
            f.language_id FROM film AS f'''



data = pd.read_sql_query(query, engine)
data.head()

OperationalError: (pymysql.err.OperationalError) (1045, "Access denied for user 'root'@'localhost' (using password: NO)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

Now let's define the query to retrive the data for our target variable (y)

In [ ]:
# retrieve and frame (y) data from sql

query_y = '''SELECT f.film_id, f.title, 
            MAX(CASE WHEN r.rental_date between '2005-05-01' AND '2005-05-31' THEN 1 ELSE 0 END) 
            AS rented_in_may FROM film AS f 
            LEFT JOIN inventory AS i ON f.film_id = i.film_id LEFT JOIN rental AS r
            ON i.inventory_id = r.inventory_id GROUP BY f.film_id, f.title'''

data_y = pd.read_sql_query(query_y, engine)
data_y.head()

In [ ]:
data_y.shape

In [ ]:
# join the dataframes

merged_data = pd.merge(data, data_y, how='left')
merged_data

In [ ]:
# check the number of May rentals

merged_data['rented_in_may'].value_counts()

I am selecting the following features for examination ['rental_duration', 'length', 'rental_rate', 'rating']

In [ ]:
# new dataframe dropping uneeded columns

selected_columns = ['rental_duration', 'length', 'rental_rate', 'rating', 'rented_in_may']
df = merged_data[selected_columns]

In [ ]:
# serach for null values

df.isna().sum()

In [ ]:
# examine the distributions of the features

features = ['rental_duration', 'length', 'rental_rate', 'rating']
for feature in features:
    plt.figure(figsize=(8,5))
    sns.histplot(merged_data[feature])
    plt.title(f'Distribution of {feature}')
    plt.show()

# exmaine the relationships of these features with 'rented_in_may'

for feature in features:
    plt.figure(figsize=(8,5))
    sns.boxplot(x='rented_in_may', y=feature, data=merged_data)
    plt.title(f'{feature} vs Rented in May')
    plt.show()

In [ ]:
# one-hot encoding for rating

df = pd.get_dummies(df, columns=['rating'], drop_first=True)


# scaling numerical variables

scaler = MinMaxScaler()
df[features[:-1]] = scaler.fit_transform(df[features[:-1]])

In [ ]:
#split the data into train and test sets

X = df[features[:-1]]  # Using all features except the last one (assuming 'rented_in_may' is the last in the list)
y = df['rented_in_may']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
# log. regression model

log_reg = LogisticRegression()

In [ ]:
#train the model

log_reg.fit(X_train, y_train)

In [ ]:
# predict on the test set
y_pred = log_reg.predict(X_test)

In [ ]:
# check the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")

# Print classification report with zero_division=1 to omit 0 vales for rented in may.

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, zero_division=1))

# heatmap of the confusion matrix

sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, cmap="YlOrRd")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

At 0.62, this tells us that when the model predicts a movie will be rented, it's right 62% of the time. We could use this information to support business decisions regarding inventory and promotions.

In situations where both FPs (mistakenly predicting a rental) and FNs (missing out on predicting a rental) have business consequences, we might need to balance things. We might be more ilkely to accept some FPs for the sake of capturing all actual rentals.